# Imports & Initialization

In [1]:
# Move to the root
import os
cwd = os.getcwd()
if os.path.basename(cwd) != "cv-in-farming":
    os.chdir("../")
print("Current directory:", os.getcwd())

Current directory: /home/cengerkin/Desktop/cv-in-farming


In [2]:
import torch

from src.dataloader import FurrowDataset
from src.model import RidgeDetector
from src.solver import Solver
from utils.helpers import save_checkpoint, load_checkpoint

optimizers = {
    "adam": torch.optim.Adam,
    "sgd": torch.optim.SGD,
    #...
}

%matplotlib inline
%load_ext autoreload
%autoreload 2

## Dataset & Dataloader

In [3]:
data_args = {
    "data_path": "dataset/20201112_140127",
    "load_darr": True,
    "load_edge": False,
    "load_rgb": False,
    "load_drgb": False,
    "load_time": False,
    "start": 0,
    "end": 100,
    "max_frames": np.inf,
}

dataset = FurrowDataset(data_args)
print(dataset)

Status of dataset:
* Dataset path: dataset/20201112_140127
* Number of frames: 101
* Load depth arrays: True
* Load edge coordinates: False
* Load RGBs: False
* Load depth RGB files: False
* Load timestamps: False



## Model & Optimizer

### Create New Model & Optimizer Instance

In [4]:
model_args = {
    "pretrained": True,
    "fuse": True,
}

optim_args = {
    "lr": 0.001,
    "betas": (0.9, 0.999),
    "eps": 1e-08,
    "weight_decay": 0,
    "amsgrad": False,
}

model = RidgeDetector(model_args)
optim = optimizers['adam'](filter(lambda p: p.requires_grad, model.parameters()), **optim_args)

print(model)
print(optim)

RidgeDetector(
  (stage1): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
  )
  (sideout1): Sequential(
    (0): Conv2d(64, 1, kernel_size=(1, 1), stride=(1, 1))
    (1): Upsample(scale_factor=1.0, mode=bilinear)
  )
  (stage2): Sequential(
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
  )
  (sideout2): Sequential(
    (0): Conv2d(128, 1, kernel_size=(1, 1), stride=(1, 1))
    (1): Upsample(scale_factor=2.0, mode=bilinear)
  )
  (stage3): Sequential(
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3

### Load Stored Model & Optimizer

In [ ]:
ckpt_path = ""
last_epoch, last_loss, last_acc, model, optim = load_checkpoint(ckpt_path)

## Solver

In [ ]:
solver_args = {
    "log_path": "/home/cengerkin/Desktop/cv-in-farming/log",
}

solver = Solver(solver_args)

print(solver)

# Train

In [ ]:
train_args = {
    "ckpt_path": "/home/cengerkin/Desktop/cv-in-farming/checkpoint",
    "max_epochs": 1,
    "val_freq": 0,
    "log_freq": 0,
    "ckpt_freq": 0,
}

# solver.train(...)

In [ ]:
save_checkpoint(ckpt_path, epoch, model, optim, loss=None, acc=None)

# Test

In [ ]:
# solver.test(...)

# Generate Video from Frames

In [ ]:
from matplotlib.animation import FFMpegWriter

plt.rcParams['animation.ffmpeg_path'] = '/usr/bin/ffmpeg'

folder = "front(20201112_140127)"
fig = plt.figure()
cut = folder_frame_map[folder][150:450]

# Fixing random state for reproducibility
metadata = dict(title='Detection Demo')
writer = FFMpegWriter(fps=30, metadata=metadata)

fig = plt.figure()
imgh = plt.imshow(np.zeros((480, 640), dtype=np.uint8))
ph, = plt.plot([], [], color="cyan", linewidth=2)

with writer.saving(fig, "Detection Demo.mp4", 100):
    for rgb_im_file, depth_arr_file, depth_im_file in cut:
        frame_idx = rgb_im_file.split("_")[0]

        rgb_im_path = os.path.join(folder, rgb_im_file)
        rgb_img = cv2.imread(rgb_im_path, cv2.IMREAD_COLOR)
        edge_pixels = np.load(os.path.join(folder, f"{frame_idx}_edge_pts.npy"))
        
        imgh.set_data(rgb_img)
        ph.set_data(edge_pixels[:,1], edge_pixels[:,0])
        
        writer.grab_frame()